In [2]:
import pandas as pd

DAY = 24*60*60

In [3]:
df = pd.read_csv('./connection_graph.csv', index_col=0, low_memory=False)
df

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon,departure_time_sec,arrival_time_sec,direction
0,A,20:52:00,20:53:00,Zajezdnia Obornicka,Paprotna,51.148737,17.021069,51.147752,17.020539,75120,75180,Paprotna
1,A,20:53:00,20:54:00,Paprotna,Obornicka (Wołowska),51.147752,17.020539,51.144385,17.023735,75180,75240,KROMERA
2,A,20:54:00,20:55:00,Obornicka (Wołowska),Bezpieczna,51.144385,17.023735,51.141360,17.026376,75240,75300,KROMERA
3,A,20:55:00,20:57:00,Bezpieczna,Bałtycka,51.141360,17.026376,51.136632,17.030617,75300,75420,pl. Daniłowskiego
4,A,20:57:00,20:59:00,Bałtycka,Broniewskiego,51.136632,17.030617,51.135851,17.037383,75420,75540,pl. Daniłowskiego
...,...,...,...,...,...,...,...,...,...,...,...,...
996209,967,18:38:00,18:39:00,Smolec - Lipowa/pętla,Smolec - Dębowa (sklep),51.075005,16.884421,51.076932,16.881543,67080,67140,FAT
996210,967,18:39:00,18:41:00,Smolec - Dębowa (sklep),Krzeptów - skrzy.,51.076932,16.881543,51.088752,16.875808,67140,67260,FAT
996211,967,18:41:00,18:42:00,Krzeptów - skrzy.,Krzeptów - Boisko,51.088752,16.875808,51.090678,16.885584,67260,67320,Smolec - Wiśniowa
996212,967,18:42:00,18:43:00,Krzeptów - Boisko,Krzeptów - Dolina Krzeptowa,51.090678,16.885584,51.092268,16.892523,67320,67380,Smolec - Wiśniowa


In [4]:
# rm the first column and duplicate rows
# df = df.drop(df.columns[0], axis=1)
df.drop_duplicates(inplace=True)
df = df[df['start_stop'] != df['end_stop']]
df

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon,departure_time_sec,arrival_time_sec,direction
0,A,20:52:00,20:53:00,Zajezdnia Obornicka,Paprotna,51.148737,17.021069,51.147752,17.020539,75120,75180,Paprotna
1,A,20:53:00,20:54:00,Paprotna,Obornicka (Wołowska),51.147752,17.020539,51.144385,17.023735,75180,75240,KROMERA
2,A,20:54:00,20:55:00,Obornicka (Wołowska),Bezpieczna,51.144385,17.023735,51.141360,17.026376,75240,75300,KROMERA
3,A,20:55:00,20:57:00,Bezpieczna,Bałtycka,51.141360,17.026376,51.136632,17.030617,75300,75420,pl. Daniłowskiego
4,A,20:57:00,20:59:00,Bałtycka,Broniewskiego,51.136632,17.030617,51.135851,17.037383,75420,75540,pl. Daniłowskiego
...,...,...,...,...,...,...,...,...,...,...,...,...
996209,967,18:38:00,18:39:00,Smolec - Lipowa/pętla,Smolec - Dębowa (sklep),51.075005,16.884421,51.076932,16.881543,67080,67140,FAT
996210,967,18:39:00,18:41:00,Smolec - Dębowa (sklep),Krzeptów - skrzy.,51.076932,16.881543,51.088752,16.875808,67140,67260,FAT
996211,967,18:41:00,18:42:00,Krzeptów - skrzy.,Krzeptów - Boisko,51.088752,16.875808,51.090678,16.885584,67260,67320,Smolec - Wiśniowa
996212,967,18:42:00,18:43:00,Krzeptów - Boisko,Krzeptów - Dolina Krzeptowa,51.090678,16.885584,51.092268,16.892523,67320,67380,Smolec - Wiśniowa


In [4]:
# df.to_csv('./connection_graph.csv')

In [5]:
def get_sec_from_str(time_str):
    time_str = time_str.split(':')
    return (int(time_str[0]) * 3600 + int(time_str[1]) * 60 + int(time_str[2])) % DAY

df['departure_time_sec'] = df['departure_time'].apply(get_sec_from_str)
df['arrival_time_sec'] = df['arrival_time'].apply(get_sec_from_str)
df

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon,departure_time_sec,arrival_time_sec,direction
0,A,20:52:00,20:53:00,Zajezdnia Obornicka,Paprotna,51.148737,17.021069,51.147752,17.020539,75120,75180,Paprotna
1,A,20:53:00,20:54:00,Paprotna,Obornicka (Wołowska),51.147752,17.020539,51.144385,17.023735,75180,75240,KROMERA
2,A,20:54:00,20:55:00,Obornicka (Wołowska),Bezpieczna,51.144385,17.023735,51.141360,17.026376,75240,75300,KROMERA
3,A,20:55:00,20:57:00,Bezpieczna,Bałtycka,51.141360,17.026376,51.136632,17.030617,75300,75420,pl. Daniłowskiego
4,A,20:57:00,20:59:00,Bałtycka,Broniewskiego,51.136632,17.030617,51.135851,17.037383,75420,75540,pl. Daniłowskiego
...,...,...,...,...,...,...,...,...,...,...,...,...
996209,967,18:38:00,18:39:00,Smolec - Lipowa/pętla,Smolec - Dębowa (sklep),51.075005,16.884421,51.076932,16.881543,67080,67140,FAT
996210,967,18:39:00,18:41:00,Smolec - Dębowa (sklep),Krzeptów - skrzy.,51.076932,16.881543,51.088752,16.875808,67140,67260,FAT
996211,967,18:41:00,18:42:00,Krzeptów - skrzy.,Krzeptów - Boisko,51.088752,16.875808,51.090678,16.885584,67260,67320,Smolec - Wiśniowa
996212,967,18:42:00,18:43:00,Krzeptów - Boisko,Krzeptów - Dolina Krzeptowa,51.090678,16.885584,51.092268,16.892523,67320,67380,Smolec - Wiśniowa


In [6]:
stops = df['start_stop'].unique()
stops, stops.shape

(array(['Zajezdnia Obornicka', 'Paprotna', 'Obornicka (Wołowska)',
        'Bezpieczna', 'Bałtycka', 'Broniewskiego', 'Pola', 'Syrokomli',
        'Kasprowicza', 'pl. Daniłowskiego', 'Przybyszewskiego',
        'Czajkowskiego', 'KOSZAROWA (Uniwersytet)', 'KOSZAROWA (Szpital)',
        'Berenta', 'KROMERA', 'Mosty Warszawskie', 'Wyszyńskiego',
        'Ogród Botaniczny', 'Katedra',
        'Urząd Wojewódzki (Muzeum Narodowe)', 'Poczta Główna',
        'skwer Krasińskiego', 'Wzgórze Partyzantów', 'Renoma',
        'Arkady (Capitol)', 'Pl. Hirszfelda', 'Krucza',
        'Krucza (Mielecka)', 'Inżynierska', 'Aleja Pracy', 'FAT',
        'GRABISZYŃSKA (Cmentarz)', 'Solskiego', 'Wiejska', 'Kadłubka',
        'Stanki', 'Bukowskiego', 'RACŁAWICKA', 'Rymarska', 'Wawrzyniaka',
        'Chłodna', 'Sowia', 'KRZYKI', 'Dworzec Główny (Dworcowa)',
        'Krasińskiego', 'Damrota', 'Koszarowa', 'Sołtysowicka',
        'Poprzeczna', 'Redycka', 'Bagatela', 'SOŁTYSOWICE', 'POŚWIĘTNE',
        'Wołowska',

In [9]:
# init:
# df['direction'] = None
# schedule = df.sort_values(by=['line', 'start_stop', 'departure_time_sec'])

/tmp/ipykernel_560175/2236890880.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['direction'] = None


In [10]:
schedule

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon,departure_time_sec,arrival_time_sec,direction
58617,1,05:11:00,05:12:00,BISKUPIN,Spółdzielcza,51.101257,17.109142,51.102206,17.102322,18660,18720,None
54380,1,05:17:00,05:18:00,BISKUPIN,Spółdzielcza,51.101257,17.109142,51.102206,17.102322,19020,19080,None
59227,1,05:26:00,05:27:00,BISKUPIN,Spółdzielcza,51.101257,17.109142,51.102206,17.102322,19560,19620,None
54536,1,05:37:00,05:38:00,BISKUPIN,Spółdzielcza,51.101257,17.109142,51.102206,17.102322,20220,20280,None
58242,1,05:41:00,05:42:00,BISKUPIN,Spółdzielcza,51.101257,17.109142,51.102206,17.102322,20460,20520,None
...,...,...,...,...,...,...,...,...,...,...,...,...
41204,N,22:58:00,23:00:00,Żmudzka,Kiełczowska,51.144008,17.140981,51.145712,17.135754,82680,82800,None
41165,N,23:08:00,23:09:00,Żmudzka,Kowieńska,51.144681,17.140957,51.142934,17.134410,83280,83340,None
41171,N,23:19:00,23:20:00,Żmudzka,Kiełczowska,51.144008,17.140981,51.145712,17.135754,83940,84000,None
41089,N,23:42:00,23:43:00,Żmudzka,Kowieńska,51.144681,17.140957,51.142934,17.134410,85320,85380,None


In [11]:
# find buses directions
def get_next_stop(current, schedule):
  return schedule[(current['end_stop'] == schedule['start_stop']) & (current['line'] == schedule['line']) & (current['start_stop'] != schedule['end_stop'])]
  
def get_previous_stop(current, schedule):
  return schedule[(current['start_stop'] == schedule['end_stop']) & (current['line'] == schedule['line']) & (current['end_stop'] != schedule['start_stop'])]

class DURATION:
  ONE_MINUTE = 24
  FIVE_MINUTES = 120
  FIFTEEN_MINUTES = 360
  HOUR = 1440
  FOREVER = len(schedule)

connections = []
id = 0
while len(schedule) > 0 and id < DURATION.FIFTEEN_MINUTES:
  current = schedule.iloc[0]
  connections.append(current)
  current = get_next_stop(current, schedule)
  
  while len(current) > 0:
    current = current.iloc[0]
    connections.append(current)

    print(current.name, end=',')
    schedule.drop(current.name, inplace=True)

    current = get_next_stop(current, schedule)
  
  direction = connections[-1]['end_stop']

  current = get_previous_stop(connections[0], schedule)
  
  while len(current) > 0:
    current = current.iloc[0]
    connections.append(current)
    
    print(current.name, end=',')
    schedule.drop(current.name, inplace=True)
    
    current = get_previous_stop(current, schedule)

  for connection in connections:
    df.loc[connection.name, 'direction'] = direction
  connections.clear()
  id += 1
  

58618,58619,58620,58621,58622,58623,58624,58625,58626,58627,58628,58629,54507,54509,54510,54511,54512,54513,54514,54381,54382,54383,54384,54385,54386,54387,54388,54389,54390,54391,54392,55440,

KeyboardInterrupt: 

In [9]:
import numpy as np

import networkx as nx

graph = nx.DiGraph()
for stop in stops:
    graph.add_node(stop)

for number, edge in df.iterrows():
  if not graph.has_edge(edge['start_stop'], edge['end_stop']):
    graph.add_edge(edge['start_stop'], edge['end_stop'], schedule=[])

  graph[edge['start_stop']] [edge['end_stop']] ['schedule'].append({
    'dep': edge['departure_time_sec'],
    'line': edge['line'],
    'direction': edge['direction'],
    'end_stop_lat': edge['end_stop_lat'],
    'end_stop_lon': edge['end_stop_lon'],
    'arrival': edge['arrival_time_sec'],
    'id': number
    })


def convert_to_dataframe(graph):
  for start, stop in graph.edges:
    graph[start][stop]['schedule'] = pd.DataFrame(graph[start][stop]['schedule'])

convert_to_dataframe(graph)

In [8]:
def print_graph(graph):
  for start, stop in graph.edges:
    print(start, '->', stop)
    print(graph[start][stop]['schedule'])
# print_graph(graph)

In [10]:
class MinProperties:
  def reset(self):
    self.min = float('inf')
    self.min_diff = float('inf')
    self.min_idx = 0

min_properties = MinProperties()  
def difference(row, departure):
    global min_properties
    arrival = row['arrival']
    new_departure = row['dep']

    # jeśli przyjechałeś o 23:55, to nie wyjedziesz o 23:50 (1)
    # jeśli przyjechałeś o 00:05, to nie wyjedziesz o 23:55 (2)
    # 
    if departure > new_departure or new_departure - departure > 20 * 60 * 60:
        return row

    value = (arrival - departure + DAY) % DAY
    # value = arrival - departure
    
    if value < min_properties.min_diff:
        min_properties.min = arrival
        min_properties.min_diff = value
        min_properties.min_idx = row['id']
    return row


In [11]:
def present_connections(visited, start, destination):
  print(f"\t\t\t\t\t\tFrom {start} to {destination}")
  for i, node in enumerate(visited):
    try:
      row = df.loc[node.id]
      print(f" {i:2d} {row['start_stop']:40s} {row['departure_time']}{' ':5s} -> {' ':5s} {row['end_stop']:40s} {row['arrival_time']:8s} linia {row['line']}")
    except:...


In [12]:
from typing import Dict

min_properties = MinProperties()

class NodeFeatures:
  def __init__(self, id, name: str, time: float, prev):
    self.id = id
    self.name = name
    self.time = time # departue time
    self.prev = prev

  def set_time(self, departure, edge, prev):
    time, id = self.get_best_time(edge, departure)
    if prev.name != prev.name and time < self.time:
      print(edge)
    if time < self.time:
      self.prev = prev
      self.id = id
      self.time = time

  def get_best_time(self, edge, departure):
    schedule = edge['schedule']

    min_properties.reset()
    schedule.apply(difference, axis=1, args=(departure,))

    return min_properties.min, min_properties.min_idx

def get_best_path(visited: Dict[str, NodeFeatures], destination: str):
  start = visited[destination]
  path = [start]

  while start.prev:
    start = start.prev
    path.append(start)

  return path[::-1]

np.seterr(divide='ignore', invalid='ignore')

def djikstra(graph: nx.Graph, start: str, destination: str, starting_time: int):
  unvisited: Dict[str, NodeFeatures] = {node: NodeFeatures(-1, node, np.inf, None) for node in graph.nodes}
  visited: Dict[str, NodeFeatures] = {}

  start_stop = unvisited.pop(start)
  start_stop.time = starting_time

  while unvisited:
    neighbours = graph[start_stop.name]
    # print("Going From", start_stop.name)
    for neigbour in neighbours:
      try:
        neigbour_node = unvisited[neigbour]
      except:
        neigbour_node = visited[neigbour]

      neigbour_node.set_time(start_stop.time, neighbours[neigbour], start_stop)
    
    visited[start_stop.name] = start_stop

    unvisited_candidates = np.array([[node.name, node.time] for node in unvisited.values()])
    min = unvisited_candidates[unvisited_candidates[:, 1].astype(float).argmin()][0]

    start_stop = unvisited.pop(min)
      
  return get_best_path(visited, destination)

START = 'PL. GRUNWALDZKI'
DESTINATION = 'Złotniki'
visited = djikstra(graph, START, DESTINATION , 8 * 60 * 60 + 50 * 60)
present_connections(visited, START, DESTINATION)

						From PL. GRUNWALDZKI to Złotniki
  1 PL. GRUNWALDZKI                          08:50:00      ->       Reja                                     08:51:00 linia 10
  2 Reja                                     08:51:00      ->       Katedra                                  08:53:00 linia 10
  3 Katedra                                  08:53:00      ->       Urząd Wojewódzki (Muzeum Narodowe)       08:54:00 linia N
  4 Urząd Wojewódzki (Muzeum Narodowe)       08:54:00      ->       GALERIA DOMINIKAŃSKA                     08:58:00 linia 2
  5 GALERIA DOMINIKAŃSKA                     08:58:00      ->       Wzgórze Partyzantów                      09:00:00 linia 2
  6 Wzgórze Partyzantów                      09:01:00      ->       Renoma                                   09:03:00 linia 149
  7 Renoma                                   09:03:00      ->       pl. Orląt Lwowskich                      09:06:00 linia 149
  8 pl. Orląt Lwowskich                      09:06:00      ->       Dworz

In [13]:
RANO =      6 * 60**2 + 30 * 60
POLUDNIE = 15 * 60**2 + 30 * 60
WIECZOR =  20 * 60**2 + 30 * 60

In [14]:
START = 'Krzeptów - Dolina Krzeptowa'
DESTINATION = 'KSIĘŻE MAŁE'
visited = djikstra(graph, START, DESTINATION , RANO)
present_connections(visited, START, DESTINATION)

						From Krzeptów - Dolina Krzeptowa to KSIĘŻE MAŁE
  1 Krzeptów - Dolina Krzeptowa              06:30:00      ->       Krzeptów - Boisko                        06:31:00 linia 927
  2 Krzeptów - Boisko                        06:31:00      ->       Krzeptów - pętla                         06:32:00 linia 927
  3 Krzeptów - pętla                         06:32:00      ->       Krzeptów - skrzy.                        06:33:00 linia 927
  4 Krzeptów - skrzy.                        06:33:00      ->       Smolec - Główna (na wys. nr 83)          06:36:00 linia 927
  5 Smolec - Główna (na wys. nr 83)          06:36:00      ->       Smolec - Chłopska/Główna                 06:38:00 linia 927
  6 Smolec - Chłopska/Główna                 06:38:00      ->       Smolec - Chłopska/Cisowa                 06:40:00 linia 927
  7 Smolec - Chłopska/Cisowa                 06:40:00      ->       Smolec - Chłopska/Śliwkowa               06:42:00 linia 927
  8 Smolec - Chłopska/Śliwkowa               06:42

In [15]:
START = 'Krzeptów - Dolina Krzeptowa'
DESTINATION = 'KSIĘŻE MAŁE'
visited = djikstra(graph, START, DESTINATION , POLUDNIE)
present_connections(visited, START, DESTINATION)

						From Krzeptów - Dolina Krzeptowa to KSIĘŻE MAŁE
  1 Krzeptów - Dolina Krzeptowa              16:12:00      ->       Krzeptów - Boisko                        16:13:00 linia 927
  2 Krzeptów - Boisko                        16:13:00      ->       Krzeptów - pętla                         16:14:00 linia 927
  3 Krzeptów - pętla                         16:14:00      ->       Krzeptów - skrzy.                        16:15:00 linia 927
  4 Krzeptów - skrzy.                        16:15:00      ->       Smolec - Główna (na wys. nr 83)          16:18:00 linia 927
  5 Smolec - Główna (na wys. nr 83)          16:18:00      ->       Smolec - Chłopska/Główna                 16:20:00 linia 927
  6 Smolec - Chłopska/Główna                 16:20:00      ->       Smolec - Chłopska/Cisowa                 16:22:00 linia 927
  7 Smolec - Chłopska/Cisowa                 16:22:00      ->       Smolec - Chłopska/Śliwkowa               16:24:00 linia 927
  8 Smolec - Chłopska/Śliwkowa               16:24

In [16]:
START = 'Krzeptów - Dolina Krzeptowa'
DESTINATION = 'KSIĘŻE MAŁE'
visited = djikstra(graph, START, DESTINATION , WIECZOR)
present_connections(visited, START, DESTINATION)

						From Krzeptów - Dolina Krzeptowa to KSIĘŻE MAŁE
  1 Krzeptów - Dolina Krzeptowa              21:12:00      ->       Krzeptów - Boisko                        21:13:00 linia 927
  2 Krzeptów - Boisko                        21:13:00      ->       Krzeptów - pętla                         21:14:00 linia 927
  3 Krzeptów - pętla                         21:14:00      ->       Krzeptów - skrzy.                        21:15:00 linia 927
  4 Krzeptów - skrzy.                        21:15:00      ->       Smolec - Główna (na wys. nr 83)          21:18:00 linia 927
  5 Smolec - Główna (na wys. nr 83)          21:18:00      ->       Smolec - Chłopska/Główna                 21:20:00 linia 927
  6 Smolec - Chłopska/Główna                 21:20:00      ->       Smolec - Chłopska/Cisowa                 21:22:00 linia 927
  7 Smolec - Chłopska/Cisowa                 21:22:00      ->       Smolec - Chłopska/Śliwkowa               21:24:00 linia 927
  8 Smolec - Chłopska/Śliwkowa               21:24

In [17]:
START = 'PL. GRUNWALDZKI'
DESTINATION = 'DWORZEC GŁÓWNY'
visited = djikstra(graph, START, DESTINATION , RANO)
present_connections(visited, START, DESTINATION)

						From PL. GRUNWALDZKI to DWORZEC GŁÓWNY
  1 PL. GRUNWALDZKI                          06:31:00      ->       most Grunwaldzki                         06:32:00 linia D
  2 most Grunwaldzki                         06:32:00      ->       Poczta Główna                            06:34:00 linia D
  3 Poczta Główna                            06:34:00      ->       skwer Krasińskiego                       06:36:00 linia A
  4 skwer Krasińskiego                       06:36:00      ->       Wzgórze Partyzantów                      06:37:00 linia A
  5 Wzgórze Partyzantów                      06:37:00      ->       DWORZEC GŁÓWNY                           06:39:00 linia 9


In [18]:
START = 'PL. GRUNWALDZKI'
DESTINATION = 'DWORZEC GŁÓWNY'
visited = djikstra(graph, START, DESTINATION , POLUDNIE)
present_connections(visited, START, DESTINATION)

						From PL. GRUNWALDZKI to DWORZEC GŁÓWNY
  1 PL. GRUNWALDZKI                          15:30:00      ->       most Grunwaldzki                         15:31:00 linia 12
  2 most Grunwaldzki                         15:32:00      ->       Poczta Główna                            15:34:00 linia D
  3 Poczta Główna                            15:34:00      ->       skwer Krasińskiego                       15:36:00 linia 145
  4 skwer Krasińskiego                       15:36:00      ->       DWORZEC GŁÓWNY                           15:38:00 linia 145


In [19]:
START = 'PL. GRUNWALDZKI'
DESTINATION = 'DWORZEC GŁÓWNY'
visited = djikstra(graph, START, DESTINATION , WIECZOR)
present_connections(visited, START, DESTINATION)

						From PL. GRUNWALDZKI to DWORZEC GŁÓWNY
  1 PL. GRUNWALDZKI                          20:30:00      ->       Reja                                     20:32:00 linia 111
  2 Reja                                     20:32:00      ->       Katedra                                  20:34:00 linia 111
  3 Katedra                                  20:35:00      ->       Urząd Wojewódzki (Muzeum Narodowe)       20:37:00 linia 10
  4 Urząd Wojewódzki (Muzeum Narodowe)       20:37:00      ->       GALERIA DOMINIKAŃSKA                     20:40:00 linia 10
  5 GALERIA DOMINIKAŃSKA                     20:41:00      ->       DWORZEC GŁÓWNY                           20:45:00 linia N


In [38]:
from typing import Dict
min_properties = MinProperties()

class NodeFeatures:
  def __init__(self, id, name: str, time: float, prev):
    self.id = id
    self.name = name
    self.cost = np.inf
    self.time = time # departue time
    self.prev = prev

  def set_time(self, departure, edge, prev):
    time, id = self.get_best_time(edge, departure)

    if time < self.time:
      self.prev = prev
      self.id = id
      self.time = time
      self.cost = time + calculate_manhatan(edge['schedule']['end_stop_lat'].iloc[0], edge['schedule']['end_stop_lon'].iloc[0])
      return id


  def get_best_time(self, edge, departure):
    schedule = edge['schedule']

    min_properties.reset()
    schedule.apply(difference, axis=1, args=(departure,))

    return min_properties.min, min_properties.min_idx

def get_best_path(visited: Dict[str, NodeFeatures], destination: str):
  start = visited[destination]
  path = [start]

  while start.prev:
    start = start.prev
    path.append(start)

  return path[::-1]

def calculate_manhatan(start_lat, start_long):
  global target_lat, target_long, TRAM_SPEED

  # return (abs(start_lat - target_lat) + abs(start_long - target_long)) * 111_000 / TRAM_SPEED 
  # return euclidean distance
  return np.sqrt((start_lat - target_lat)**2 + (start_long - target_long)**2) * 111_000 / TRAM_SPEED 

np.seterr(divide='ignore', invalid='ignore')

TRAM_SPEED = 16.2 * 1000 / 3600
target_lat = None
target_lat = None

def a_star(graph: nx.Graph, start: str, destination: str, starting_time: int):
  global target_long, target_lat
  target_lat = df[df['start_stop'] == destination].iloc[0]['start_stop_lat']
  target_long = df[df['start_stop'] == destination].iloc[0]['start_stop_lon']

  unvisited: Dict[str, NodeFeatures] = {node: NodeFeatures(-1, node, np.inf, None) for node in graph.nodes}
  visited: Dict[str, NodeFeatures] = {}

  start_stop = unvisited.pop(start)
  start_stop.time = starting_time

  while unvisited:
    neighbours = graph[start_stop.name]
    for neigbour in neighbours:
      try:
        neigbour_node = unvisited[neigbour]
      except:
        neigbour_node = visited[neigbour]

      neigbour_node.set_time(start_stop.time, neighbours[neigbour], start_stop)

    visited[start_stop.name] = start_stop

    unvisited_candidates = np.array([[node.name, node.cost] for node in unvisited.values()])
    min = unvisited_candidates[unvisited_candidates[:, 1].astype(float).argmin()][0]

    start_stop = unvisited.pop(min)
    # performance boost
    if start_stop.name == destination: 
      visited[start_stop.name] = start_stop
      break

  return get_best_path(visited, destination)

# visited = a_star(graph, 'PL. GRUNWALDZKI','Złotniki' , 8 * 60 * 60 + 50 * 60)
# visited

In [21]:
START = 'Krzeptów - Dolina Krzeptowa'
DESTINATION = 'KSIĘŻE MAŁE'
visited = a_star(graph, START, DESTINATION , RANO)
present_connections(visited, START, DESTINATION)

						From Krzeptów - Dolina Krzeptowa to KSIĘŻE MAŁE
  1 Krzeptów - Dolina Krzeptowa              06:30:00      ->       Krzeptów - Boisko                        06:31:00 linia 927
  2 Krzeptów - Boisko                        06:31:00      ->       Krzeptów - pętla                         06:32:00 linia 927
  3 Krzeptów - pętla                         06:32:00      ->       Krzeptów - skrzy.                        06:33:00 linia 927
  4 Krzeptów - skrzy.                        06:33:00      ->       Smolec - Główna (na wys. nr 83)          06:36:00 linia 927
  5 Smolec - Główna (na wys. nr 83)          06:36:00      ->       Smolec - Chłopska/Główna                 06:38:00 linia 927
  6 Smolec - Chłopska/Główna                 06:38:00      ->       Smolec - Chłopska/Cisowa                 06:40:00 linia 927
  7 Smolec - Chłopska/Cisowa                 06:40:00      ->       Smolec - Chłopska/Śliwkowa               06:42:00 linia 927
  8 Smolec - Chłopska/Śliwkowa               06:42

In [39]:
START = 'Krzeptów - Dolina Krzeptowa'
DESTINATION = 'KSIĘŻE MAŁE'
visited = a_star(graph, START, DESTINATION , POLUDNIE)
present_connections(visited, START, DESTINATION)

						From Krzeptów - Dolina Krzeptowa to KSIĘŻE MAŁE
  1 Krzeptów - Dolina Krzeptowa              16:12:00      ->       Krzeptów - Boisko                        16:13:00 linia 927
  2 Krzeptów - Boisko                        16:13:00      ->       Krzeptów - pętla                         16:14:00 linia 927
  3 Krzeptów - pętla                         16:14:00      ->       Krzeptów - skrzy.                        16:15:00 linia 927
  4 Krzeptów - skrzy.                        16:15:00      ->       Smolec - Główna (na wys. nr 83)          16:18:00 linia 927
  5 Smolec - Główna (na wys. nr 83)          16:18:00      ->       Smolec - Chłopska/Główna                 16:20:00 linia 927
  6 Smolec - Chłopska/Główna                 16:20:00      ->       Smolec - Chłopska/Cisowa                 16:22:00 linia 927
  7 Smolec - Chłopska/Cisowa                 16:22:00      ->       Smolec - Chłopska/Śliwkowa               16:24:00 linia 927
  8 Smolec - Chłopska/Śliwkowa               16:24

In [40]:
START = 'Krzeptów - Dolina Krzeptowa'
DESTINATION = 'KSIĘŻE MAŁE'
visited = a_star(graph, START, DESTINATION , WIECZOR)
present_connections(visited, START, DESTINATION)

						From Krzeptów - Dolina Krzeptowa to KSIĘŻE MAŁE
  1 Krzeptów - Dolina Krzeptowa              21:12:00      ->       Krzeptów - Boisko                        21:13:00 linia 927
  2 Krzeptów - Boisko                        21:13:00      ->       Krzeptów - pętla                         21:14:00 linia 927
  3 Krzeptów - pętla                         21:14:00      ->       Krzeptów - skrzy.                        21:15:00 linia 927
  4 Krzeptów - skrzy.                        21:15:00      ->       Smolec - Główna (na wys. nr 83)          21:18:00 linia 927
  5 Smolec - Główna (na wys. nr 83)          21:18:00      ->       Smolec - Chłopska/Główna                 21:20:00 linia 927
  6 Smolec - Chłopska/Główna                 21:20:00      ->       Smolec - Chłopska/Cisowa                 21:22:00 linia 927
  7 Smolec - Chłopska/Cisowa                 21:22:00      ->       Smolec - Chłopska/Śliwkowa               21:24:00 linia 927
  8 Smolec - Chłopska/Śliwkowa               21:24

In [42]:
START = 'PL. GRUNWALDZKI'
DESTINATION = 'DWORZEC GŁÓWNY'
visited = a_star(graph, START, DESTINATION , RANO)
present_connections(visited, START, DESTINATION)

						From PL. GRUNWALDZKI to DWORZEC GŁÓWNY
  1 PL. GRUNWALDZKI                          06:31:00      ->       most Grunwaldzki                         06:32:00 linia D
  2 most Grunwaldzki                         06:32:00      ->       Poczta Główna                            06:34:00 linia D
  3 Poczta Główna                            06:34:00      ->       skwer Krasińskiego                       06:36:00 linia A
  4 skwer Krasińskiego                       06:36:00      ->       Wzgórze Partyzantów                      06:37:00 linia A
  5 Wzgórze Partyzantów                      06:37:00      ->       DWORZEC GŁÓWNY                           06:39:00 linia 9


In [43]:
START = 'PL. GRUNWALDZKI'
DESTINATION = 'DWORZEC GŁÓWNY'
visited = a_star(graph, START, DESTINATION , POLUDNIE)
present_connections(visited, START, DESTINATION)

						From PL. GRUNWALDZKI to DWORZEC GŁÓWNY
  1 PL. GRUNWALDZKI                          15:30:00      ->       most Grunwaldzki                         15:31:00 linia 12
  2 most Grunwaldzki                         15:32:00      ->       Poczta Główna                            15:34:00 linia D
  3 Poczta Główna                            15:34:00      ->       skwer Krasińskiego                       15:36:00 linia 145
  4 skwer Krasińskiego                       15:36:00      ->       DWORZEC GŁÓWNY                           15:38:00 linia 145


In [44]:
START = 'PL. GRUNWALDZKI'
DESTINATION = 'DWORZEC GŁÓWNY'
visited = a_star(graph, START, DESTINATION , WIECZOR)
present_connections(visited, START, DESTINATION)

						From PL. GRUNWALDZKI to DWORZEC GŁÓWNY
  1 PL. GRUNWALDZKI                          20:30:00      ->       Reja                                     20:32:00 linia 111
  2 Reja                                     20:32:00      ->       Katedra                                  20:34:00 linia 111
  3 Katedra                                  20:35:00      ->       Urząd Wojewódzki (Muzeum Narodowe)       20:37:00 linia 10
  4 Urząd Wojewódzki (Muzeum Narodowe)       20:37:00      ->       GALERIA DOMINIKAŃSKA                     20:40:00 linia 10
  5 GALERIA DOMINIKAŃSKA                     20:41:00      ->       DWORZEC GŁÓWNY                           20:45:00 linia N


In [13]:
from typing import Dict

from typing import Dict
min_properties = MinProperties()

class NodeFeatures:
  def __init__(self, id, name: str, time: float, prev):
    self.id = id
    self.name = name
    self.cost = np.inf
    self.time = time # departue time
    self.prev = prev

  def set_time(self, departure, edge, prev):
    time, id = self.get_best_time(edge, departure)

    if time < self.time:
      self.prev = prev
      self.id = id
      self.time = time
      self.cost = time + calculate_manhatan(edge['schedule']['end_stop_lat'].iloc[0], edge['schedule']['end_stop_lon'].iloc[0])
      return id


  def get_best_time(self, edge, departure):
    schedule = edge['schedule']

    min_properties.reset()
    schedule.apply(difference, axis=1, args=(departure,))

    return min_properties.min, min_properties.min_idx

def get_best_path(visited: Dict[str, NodeFeatures], destination: str):
  start = visited[destination]
  path = [start]

  while start.prev:
    start = start.prev
    path.append(start)

  return path[::-1]

def calculate_manhatan(start_lat, start_long):
  global target_lat, target_long, TRAM_SPEED

  return (abs(start_lat - target_lat) + abs(start_long - target_long)) * 111_000 / TRAM_SPEED 

np.seterr(divide='ignore', invalid='ignore')

TRAM_SPEED = 16.2 * 1000 / 3600
target_lat = None
target_lat = None

def a_star(graph: nx.Graph, start: str, destination: str, starting_time: int):
  global target_long, target_lat
  target_lat = df[df['start_stop'] == destination].iloc[0]['start_stop_lat']
  target_long = df[df['start_stop'] == destination].iloc[0]['start_stop_lon']

  unvisited: Dict[str, NodeFeatures] = {node: NodeFeatures(-1, node, np.inf, None) for node in graph.nodes}
  visited: Dict[str, NodeFeatures] = {}

  start_stop = unvisited.pop(start)
  start_stop.time = starting_time

  while unvisited:
    neighbours = graph[start_stop.name]
    for neigbour in neighbours:
      try:
        neigbour_node = unvisited[neigbour]
      except:
        neigbour_node = visited[neigbour]

      neigbour_node.set_time(start_stop.time, neighbours[neigbour], start_stop)

    visited[start_stop.name] = start_stop

    unvisited_candidates = np.array([[node.name, node.cost] for node in unvisited.values()])
    min = unvisited_candidates[unvisited_candidates[:, 1].astype(float).argmin()][0]

    start_stop = unvisited.pop(min)
    # performance boost
    if start_stop.name == destination: 
      visited[start_stop.name] = start_stop
      break

  return get_best_path(visited, destination)

START = 'PL. GRUNWALDZKI'
DESTINATION = 'Złotniki'
visited = a_star(graph, START,DESTINATION , 8 * 60 * 60 + 50 * 60)
present_connections(visited, START, DESTINATION)

						From PL. GRUNWALDZKI to Złotniki
  1 PL. GRUNWALDZKI                          08:50:00      ->       Reja                                     08:51:00 linia 10
  2 Reja                                     08:51:00      ->       Katedra                                  08:53:00 linia 10
  3 Katedra                                  08:53:00      ->       Ogród Botaniczny                         08:55:00 linia A
  4 Ogród Botaniczny                         08:55:00      ->       pl. Bema                                 08:57:00 linia 17
  5 pl. Bema                                 09:01:00      ->       Dubois                                   09:03:00 linia 19
  6 Dubois                                   09:03:00      ->       Pomorska                                 09:05:00 linia 19
  7 Pomorska                                 09:06:00      ->       pl. Strzelecki                           09:08:00 linia 142
  8 pl. Strzelecki                           09:08:00      ->       Klec

## A* with the use of a direction

In [24]:
directions = df[['direction', 'end_stop_lat',	'end_stop_lon']].groupby('direction').first().reset_index()
directions

,direction,end_stop_lat,end_stop_lon
0,8 Maja,51.113552,17.092291
1,AUCHAN,51.059746,16.974564
2,Arkady (Capitol),51.066438,16.974130
3,Armii Krajowej (Bogedaina),51.151432,17.017385
4,Awicenny,51.081201,16.967704
...,...,...,...
399,Świstackiego,51.051113,17.092426
400,Świętokrzyska,51.117297,17.044186
401,ŻAR,51.143306,16.866378
402,Żernicka,51.127522,16.900596


In [41]:
current_lat, current_lon = (1, 1)
direction_lat, direction_log = (1+ np.sqrt(3), 1)
destination_lat, destination_lon = (1+ np.sqrt(3), 2)

# # angle = atan2(vector2.y, vector2.x) - atan2(vector1.y, vector1.x);
# angle = np.arctan2(P1[1] - P0[1], P1[0] - P0[0]) - np.arctan2(P2[1] - P0[1], P2[0] - P0[0])
angle = abs(np.degrees(np.arctan2(direction_lat - current_lat, direction_log - current_lon) - np.arctan2(destination_lat - current_lat, destination_lon - current_lon)))
angle

30.000000000000004

In [43]:
from typing import Dict

from typing import Dict
min_properties = MinProperties()
destination_lat, destination_lon = None, None

class NodeFeatures:
  def __init__(self, id, name: str, time: float, prev):
    self.id = id
    self.name = name
    self.cost = np.inf
    self.time = time # departue time
    self.prev = prev
    self.line = None

  def set_time(self, departure, edge, prev):
    time, id = self.get_best_time(edge, departure)

    if time < self.time:
      manhatan_distance = calculate_manhatan(edge['schedule']['end_stop_lat'].iloc[0], edge['schedule']['end_stop_lon'].iloc[0])
      if prev.line == edge['schedule']['line'].iloc[0]:
        line_penalty = 0
      else:
        line_penalty = 1
      angle_penalty = calculate_angle(df.loc[id]['direction'], edge['schedule']['end_stop_lat'].iloc[0], edge['schedule']['end_stop_lon'].iloc[0])
      self.prev = prev
      self.id = id
      self.time = time

      self.cost = time + manhatan_distance + line_penalty + angle_penalty
      return id


  def get_best_time(self, edge, departure):
    schedule = edge['schedule']

    min_properties.reset()
    schedule.apply(difference, axis=1, args=(departure,))

    return min_properties.min, min_properties.min_idx

def get_best_path(visited: Dict[str, NodeFeatures], destination: str):
  start = visited[destination]
  path = [start]

  while start.prev:
    start = start.prev
    path.append(start)

  return path[::-1]

def calculate_manhatan(start_lat, start_long):
  global target_lat, target_long, TRAM_SPEED
  return 0
  return (abs(start_lat - target_lat) + abs(start_long - target_long)) * 111_000 / TRAM_SPEED 

def calculate_angle(bus_direction, current_lat, current_lon):
  direction_lat, direction_lon = df[df['end_stop'] == bus_direction][['end_stop_lat', 'end_stop_lon']].iloc[0]
  angle = abs(np.degrees(np.arctan2(direction_lat - current_lat, direction_lon - current_lon) - np.arctan2(destination_lat - current_lat, destination_lon - current_lon)))
  print('angle:', angle)
  if angle < 1:
    return -1
  if angle < 5:
    return 0
  
  if angle < 10:
    return 1
  
  if angle < 20:
    return 3

  return 5

np.seterr(divide='ignore', invalid='ignore')

TRAM_SPEED = 16.2 * 1000 / 3600
target_lat = None
target_lat = None

def a_star(graph: nx.Graph, start: str, destination: str, starting_time: int):
  global target_long, target_lat
  target_lat = df[df['start_stop'] == destination].iloc[0]['start_stop_lat']
  target_long = df[df['start_stop'] == destination].iloc[0]['start_stop_lon']

  unvisited: Dict[str, NodeFeatures] = {node: NodeFeatures(-1, node, np.inf, None) for node in graph.nodes}
  visited: Dict[str, NodeFeatures] = {}

  start_stop = unvisited.pop(start)
  start_stop.time = starting_time

  while unvisited:
    neighbours = graph[start_stop.name]
    for neigbour in neighbours:
      try:
        neigbour_node = unvisited[neigbour]
      except:
        neigbour_node = visited[neigbour]

      neigbour_node.set_time(start_stop.time, neighbours[neigbour], start_stop)

    visited[start_stop.name] = start_stop

    unvisited_candidates = np.array([[node.name, node.cost] for node in unvisited.values()])
    min = unvisited_candidates[unvisited_candidates[:, 1].astype(float).argmin()][0]

    start_stop = unvisited.pop(min)
    # performance boost
    if start_stop.name == destination: 
      visited[start_stop.name] = start_stop
      break

  return get_best_path(visited, destination)

START = 'PL. GRUNWALDZKI'
DESTINATION = 'Złotniki'

destination_lat, destination_lon = df[df['end_stop'] == DESTINATION][['end_stop_lat', 'end_stop_lon']].iloc[0]

visited = a_star(graph, START,DESTINATION , 8 * 60 * 60 + 50 * 60)

present_connections(visited, START, DESTINATION)

angle: 244.93121868905442
angle: 52.89369648398297
angle: 180.78874512305808
angle: 29.649460435114882
angle: 0.6622339109173335
angle: 162.55407140272348
angle: 0.6768868885355932
angle: 172.77166720043132
angle: 234.21700685074902
angle: 44.90830857488427
angle: 178.967030952555
angle: 178.0686182920085
angle: 169.34643606566638
angle: 300.8554556546669
angle: 165.5474629921988
angle: 303.19047020031206
angle: 45.87021922796913
angle: 227.82192100200942
angle: 177.55627284349652
angle: 172.52715023920302
angle: 182.4631224542783
angle: 167.66211371995038
angle: 296.6430080268893
angle: 174.78004760423434
angle: 197.31388070270432
angle: 7.050829132564771
angle: 138.5417038815554
angle: 173.35438692065608
angle: 171.13911143165305
angle: 28.328641465142653
angle: 176.7415562549862
angle: 186.1989927891914
angle: 237.19361061646308
angle: 177.7151575011192
angle: 203.79897601813227
angle: 219.68789330411835
angle: 172.5930378694222
angle: 3.7016219106580155
angle: 169.15331504287235
an

In [45]:
from typing import Dict

from typing import Dict
min_properties = MinProperties()
destination_lat, destination_lon = None, None

class NodeFeatures:
  def __init__(self, id, name: str, time: float, prev):
    self.id = id
    self.name = name
    self.cost = np.inf
    self.time = time # departue time
    self.prev = prev
    self.line = None

  def set_time(self, departure, edge, prev):
    time, id = self.get_best_time(edge, departure)

    if time < self.time:
      manhatan_distance = calculate_manhatan(edge['schedule']['end_stop_lat'].iloc[0], edge['schedule']['end_stop_lon'].iloc[0])
      if prev.line == edge['schedule']['line'].iloc[0]:
        line_penalty = 0
      else:
        line_penalty = 1
      angle_penalty = calculate_angle(df.loc[id]['direction'], edge['schedule']['end_stop_lat'].iloc[0], edge['schedule']['end_stop_lon'].iloc[0])
      self.prev = prev
      self.id = id
      self.time = time

      self.cost = time + manhatan_distance * 0.5 + line_penalty + angle_penalty
      return id


  def get_best_time(self, edge, departure):
    schedule = edge['schedule']

    min_properties.reset()
    schedule.apply(difference, axis=1, args=(departure,))

    return min_properties.min, min_properties.min_idx

def get_best_path(visited: Dict[str, NodeFeatures], destination: str):
  start = visited[destination]
  path = [start]

  while start.prev:
    start = start.prev
    path.append(start)

  return path[::-1]

def calculate_manhatan(start_lat, start_long):
  global target_lat, target_long, TRAM_SPEED
  return (abs(start_lat - target_lat) + abs(start_long - target_long)) * 111_000 / TRAM_SPEED 

def calculate_angle(bus_direction, current_lat, current_lon):
  direction_lat, direction_lon = df[df['end_stop'] == bus_direction][['end_stop_lat', 'end_stop_lon']].iloc[0]
  angle = abs(np.degrees(np.arctan2(direction_lat - current_lat, direction_lon - current_lon) - np.arctan2(destination_lat - current_lat, destination_lon - current_lon)))
  print('angle:', angle)
  if angle < 1:
    return -1
  if angle < 5:
    return 0
  
  if angle < 10:
    return 1
  
  if angle < 20:
    return 3

  return 5

np.seterr(divide='ignore', invalid='ignore')

TRAM_SPEED = 16.2 * 1000 / 3600
target_lat = None
target_lat = None

def a_star(graph: nx.Graph, start: str, destination: str, starting_time: int):
  global target_long, target_lat
  target_lat = df[df['start_stop'] == destination].iloc[0]['start_stop_lat']
  target_long = df[df['start_stop'] == destination].iloc[0]['start_stop_lon']

  unvisited: Dict[str, NodeFeatures] = {node: NodeFeatures(-1, node, np.inf, None) for node in graph.nodes}
  visited: Dict[str, NodeFeatures] = {}

  start_stop = unvisited.pop(start)
  start_stop.time = starting_time

  while unvisited:
    neighbours = graph[start_stop.name]
    for neigbour in neighbours:
      try:
        neigbour_node = unvisited[neigbour]
      except:
        neigbour_node = visited[neigbour]

      neigbour_node.set_time(start_stop.time, neighbours[neigbour], start_stop)

    visited[start_stop.name] = start_stop

    unvisited_candidates = np.array([[node.name, node.cost] for node in unvisited.values()])
    min = unvisited_candidates[unvisited_candidates[:, 1].astype(float).argmin()][0]

    start_stop = unvisited.pop(min)
    # performance boost
    if start_stop.name == destination: 
      visited[start_stop.name] = start_stop
      break

  return get_best_path(visited, destination)

START = 'PL. GRUNWALDZKI'
DESTINATION = 'Złotniki'

destination_lat, destination_lon = df[df['end_stop'] == DESTINATION][['end_stop_lat', 'end_stop_lon']].iloc[0]

visited = a_star(graph, START,DESTINATION , 8 * 60 * 60 + 50 * 60)

present_connections(visited, START, DESTINATION)

angle: 244.93121868905442
angle: 52.89369648398297
angle: 180.78874512305808
angle: 29.649460435114882
angle: 0.6622339109173335
angle: 162.55407140272348
angle: 0.6768868885355932
angle: 169.34643606566638
angle: 300.8554556546669
angle: 172.77166720043132
angle: 234.21700685074902
angle: 303.19047020031206
angle: 45.87021922796913
angle: 296.6430080268893
angle: 174.78004760423434
angle: 197.31388070270432
angle: 7.050829132564771
angle: 163.89641441306404
angle: 10.263623297934844
angle: 224.72918869697062
angle: 227.82192100200942
angle: 166.60585006702004
angle: 297.3644617567564
angle: 192.72087158931473
angle: 44.54680497777794
angle: 52.753992085038014
angle: 178.967030952555
angle: 178.0686182920085
angle: 237.19361061646308
angle: 305.6676937378296
angle: 165.5474629921988
angle: 65.51412827677129
angle: 173.76894245105402
angle: 28.328641465142653
angle: 173.49185401393927
angle: 221.5726617090628
angle: 316.0978263419659
angle: 221.87348169330346
angle: 0.2740585776835347
a

## TABU

In [28]:
DESTINATION = 'Złotniki'

'KROMERA'

In [49]:
elements = ['Tyrmanda', 'Młodych Techników', 'FAT', 'Złotniki' , 'PL. GRUNWALDZKI', 'Kwiska', ]

all_permutations = [np.random.permutation(elements) for _ in range(20)]
np.unique(all_permutations, axis=0)[:4]

array([['DWORZEC AUTOBUSOWY', 'FAT', 'GALERIA DOMINIKAŃSKA'],
       ['DWORZEC AUTOBUSOWY', 'GALERIA DOMINIKAŃSKA', 'FAT'],
       ['FAT', 'DWORZEC AUTOBUSOWY', 'GALERIA DOMINIKAŃSKA'],
       ['FAT', 'GALERIA DOMINIKAŃSKA', 'DWORZEC AUTOBUSOWY']],
      dtype='<U20')

In [50]:
for permutation in np.unique(all_permutations, axis=0):
  start_time = 34900
  end_time = start_time
  for i, stop in enumerate(permutation):
    destination = permutation[(i + 1) % 3]
    visited = a_star(graph, stop, destination, end_time)
    end_time = df.loc[visited[-1].id]['arrival_time_sec']

    print('For: ', stop, 'to:', destination, 'time:', (end_time - start_time) / 60, 'minutes')

  print('For: ', permutation, 'time:', (end_time - start_time) / 60, 'minutes')

For:  DWORZEC AUTOBUSOWY to: FAT time: 22.333333333333332 minutes
For:  FAT to: GALERIA DOMINIKAŃSKA time: 40.333333333333336 minutes
For:  GALERIA DOMINIKAŃSKA to: DWORZEC AUTOBUSOWY time: 50.333333333333336 minutes
For:  ['DWORZEC AUTOBUSOWY' 'FAT' 'GALERIA DOMINIKAŃSKA'] time: 50.333333333333336 minutes
For:  DWORZEC AUTOBUSOWY to: GALERIA DOMINIKAŃSKA time: 6.333333333333333 minutes
For:  GALERIA DOMINIKAŃSKA to: FAT time: 26.333333333333332 minutes
For:  FAT to: DWORZEC AUTOBUSOWY time: 42.333333333333336 minutes
For:  ['DWORZEC AUTOBUSOWY' 'GALERIA DOMINIKAŃSKA' 'FAT'] time: 42.333333333333336 minutes
For:  FAT to: DWORZEC AUTOBUSOWY time: 19.333333333333332 minutes
For:  DWORZEC AUTOBUSOWY to: GALERIA DOMINIKAŃSKA time: 27.333333333333332 minutes
For:  GALERIA DOMINIKAŃSKA to: FAT time: 50.333333333333336 minutes
For:  ['FAT' 'DWORZEC AUTOBUSOWY' 'GALERIA DOMINIKAŃSKA'] time: 50.333333333333336 minutes
For:  FAT to: GALERIA DOMINIKAŃSKA time: 20.333333333333332 minutes
For:  GAL

KeyboardInterrupt: 

In [45]:
from itertools import cycle
from random import randint
def set_neighbors(solution, neighbors):
    neighbors[:, :] = solution
    swappable = cycle(range(1, neighbors.shape[1] - 1, randint(1, neighbors.shape[1] - 2)))
    for i in range(0, neighbors.shape[0] - 1):
        id = next(swappable)
        neighbors[i][id], neighbors[i][id + 1] = solution[id + 1], solution[id]
    

best_solution = np.array(['Tyrmanda', 'FAT', 'Kwiska', 'Złotniki', 'PL. GRUNWALDZKI', 'Młodych Techników','Tyrmanda'])
neighborhood = np.zeros((4, len(best_solution)), dtype=object)
set_neighbors(best_solution, neighborhood)
set_neighbors(best_solution, neighborhood)
neighborhood

array([['Tyrmanda', 'Kwiska', 'FAT', 'Złotniki', 'PL. GRUNWALDZKI',
        'Młodych Techników', 'Tyrmanda'],
       ['Tyrmanda', 'FAT', 'Złotniki', 'Kwiska', 'PL. GRUNWALDZKI',
        'Młodych Techników', 'Tyrmanda'],
       ['Tyrmanda', 'FAT', 'Kwiska', 'PL. GRUNWALDZKI', 'Złotniki',
        'Młodych Techników', 'Tyrmanda'],
       ['Tyrmanda', 'FAT', 'Kwiska', 'Złotniki', 'PL. GRUNWALDZKI',
        'Młodych Techników', 'Tyrmanda']], dtype=object)

In [46]:
def objective_function(solution = ['Kwiska', 'FAT', 'PL. GRUNWALDZKI','Złotniki',  'Młodych Techników', 'Kwiska' ], start_time=9*60*60 + 40*60):
    end_time = start_time
    for i, stop in enumerate(solution[:-1]):
        destination = solution[(i + 1) ]
        visited = a_star(graph, stop, destination, end_time)
        end_time = df.loc[visited[-1].id]['arrival_time_sec']
        # print('For: ', stop, 'to:', destination, 'time:', (end_time - start_time) / 60, 'minutes')
    print(f"For: {' -> '.join(solution)} time: {(end_time - start_time) / 60} minutes")
    
    return end_time - start_time
# objective_function()

In [47]:
def add_to_tabu_list(tabu_list, tabu_idx, neighbors, should_pop_from_tabu):
    tabu_list[tabu_idx: tabu_idx + len(neighbors), :] = neighbors
    tabu_idx += len(neighbors)
    if tabu_idx >= len(tabu_list):
        tabu_idx = 0
        should_pop_from_tabu = True
    return tabu_list, tabu_idx, should_pop_from_tabu

def tabu_search(stops, start_time, max_iter=150, tabu_size=100):
    best_solution = np.array([*stops, stops[0]])

    tabu_size -= tabu_size % 4
    tabu_list = np.zeros((tabu_size, len(best_solution)), dtype=object)
    tabu_idx = 0
    should_pop_from_tabu = False
    should_add_to_tabu = False

    poor_solutions = []

    best_cost = np.inf
    neighborhood = np.zeros((4, len(best_solution)), dtype=object)
    
    for i in range(max_iter):
        set_neighbors(best_solution, neighborhood)
        
        for neighbor in neighborhood:
            # print("trynna ", neighbor)
            if not np.any(np.all(tabu_list == neighbor, axis=1)) and not neighbor.tolist() in poor_solutions:
                # print('current solution:', neighbor)
                cost = objective_function(neighbor, start_time)
                if cost < best_cost:
                    best_cost = cost
                    best_solution[:] = neighbor
                    should_add_to_tabu = True

            else: ...
                # print('tabu solution:', neighbor)
                
        if should_add_to_tabu:
            should_add_to_tabu = False
            if should_pop_from_tabu:
                tabu_list[tabu_idx: tabu_idx + len(neighborhood), :] = 0
                
            tabu_list, tabu_idx, should_pop_from_tabu = add_to_tabu_list(tabu_list, tabu_idx, neighborhood, should_pop_from_tabu)
        else:
            poor_solutions.extend(neighborhood.tolist())
            
    return best_solution, best_cost

In [ ]:
stops = ['Tyrmanda', 'Młodych Techników', 'FAT', 'Złotniki' , 'PL. GRUNWALDZKI', 'Kwiska', ]
best_solution, best_cost = tabu_search(stops, start_time=8*60*60 + 40*60)
print(f"The best result: {' -> '.join(best_solution)} time: {best_cost / 60} minutes")

For: Tyrmanda -> FAT -> Młodych Techników -> Złotniki -> PL. GRUNWALDZKI -> Kwiska -> Tyrmanda time: 157.0 minutes
For: Tyrmanda -> Młodych Techników -> FAT -> Złotniki -> Kwiska -> PL. GRUNWALDZKI -> Tyrmanda time: 189.0 minutes
For: Tyrmanda -> FAT -> Młodych Techników -> Złotniki -> PL. GRUNWALDZKI -> Kwiska -> Tyrmanda time: 157.0 minutes
For: Tyrmanda -> Młodych Techników -> FAT -> Złotniki -> PL. GRUNWALDZKI -> Kwiska -> Tyrmanda time: 189.0 minutes
For: Tyrmanda -> FAT -> Młodych Techników -> Złotniki -> Kwiska -> PL. GRUNWALDZKI -> Tyrmanda time: 159.0 minutes
For: Tyrmanda -> FAT -> Złotniki -> Młodych Techników -> PL. GRUNWALDZKI -> Kwiska -> Tyrmanda time: 129.0 minutes
For: Tyrmanda -> FAT -> Młodych Techników -> PL. GRUNWALDZKI -> Złotniki -> Kwiska -> Tyrmanda time: 153.0 minutes
For: Tyrmanda -> Złotniki -> FAT -> Młodych Techników -> PL. GRUNWALDZKI -> Kwiska -> Tyrmanda time: 129.0 minutes
For: Tyrmanda -> FAT -> Złotniki -> PL. GRUNWALDZKI -> Młodych Techników -> Kwis

In [48]:
stops = ['Tyrmanda', 'BISKUPIN', 'PL. GRUNWALDZKI', 'Kwiska', 'KSIĘŻE MAŁE']
best_solution, best_cost = tabu_search(stops, start_time=8*60*60 + 40*60)
print(f"The best result: {' -> '.join(best_solution)} time: {best_cost / 60} minutes")

For: Tyrmanda -> PL. GRUNWALDZKI -> BISKUPIN -> Kwiska -> KSIĘŻE MAŁE -> Tyrmanda time: 175.0 minutes
For: Tyrmanda -> BISKUPIN -> PL. GRUNWALDZKI -> Kwiska -> Tyrmanda -> KSIĘŻE MAŁE time: 151.0 minutes
For: Tyrmanda -> PL. GRUNWALDZKI -> BISKUPIN -> Kwiska -> KSIĘŻE MAŁE -> Tyrmanda time: 175.0 minutes
For: Tyrmanda -> BISKUPIN -> PL. GRUNWALDZKI -> Kwiska -> KSIĘŻE MAŁE -> Tyrmanda time: 175.0 minutes
For: Tyrmanda -> PL. GRUNWALDZKI -> BISKUPIN -> Kwiska -> Tyrmanda -> KSIĘŻE MAŁE time: 151.0 minutes
For: Tyrmanda -> PL. GRUNWALDZKI -> BISKUPIN -> Kwiska -> Tyrmanda -> KSIĘŻE MAŁE time: 151.0 minutes
For: Tyrmanda -> BISKUPIN -> PL. GRUNWALDZKI -> Tyrmanda -> Kwiska -> KSIĘŻE MAŁE time: 151.0 minutes
For: Tyrmanda -> BISKUPIN -> Kwiska -> PL. GRUNWALDZKI -> Tyrmanda -> KSIĘŻE MAŁE time: 191.0 minutes
The best result: Tyrmanda -> BISKUPIN -> PL. GRUNWALDZKI -> Kwiska -> Tyrmanda -> KSIĘŻE MAŁE time: 151.0 minutes
